In [2]:
!pip install google-generativeai tavily-python python-dotenv rich langgraph langchain langchain-core


Imports & Load Environment Variables

In [1]:
import os
from dotenv import load_dotenv
from rich.console import Console
from rich.panel import Panel
from rich.markdown import Markdown
from datetime import datetime
from typing import List

load_dotenv()
console = Console()


Initialize Gemini

In [2]:
import google.generativeai as genai

if not os.getenv("GEMINI_API_KEY"):
    raise ValueError(" GEMINI_API_KEY not found in .env file.")

genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
model = genai.GenerativeModel("gemini-1.5-flash")

console.print("[green]✅ Gemini model initialized[/green]")


✅ Gemini model initialized

Initialize Tavily Search

In [3]:
from tavily import TavilyClient

if not os.getenv("TAVILY_API_KEY"):
    raise ValueError("❌ TAVILY_API_KEY not found in .env file.")

tavily = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

def tavily_search(query: str) -> str:
    result = tavily.search(query=query, max_results=5)
    text = "\n".join([f"- {r['title']}: {r['url']}" for r in result["results"]])
    return text

console.print("[green]✅ Tavily Search ready[/green]")


✅ Tavily Search ready

Research Agent Core Workflow

In [5]:
def clarify_if_needed(user_query: str) -> str:
    prompt = f"""
You are a research assistant. Determine if the user query is specific enough.
If NOT, ask exactly ONE clarifying question.
If yes, return "clear".
User query: "{user_query}"
"""
    response = model.generate_content(prompt).text
    return response

def produce_research_brief(user_query: str) -> str:
    prompt = f"""
Convert this user query into a clear, specific research question.
User query: "{user_query}"
Return ONLY the research question.
"""
    return model.generate_content(prompt).text.strip()

def iterative_research(research_question: str):
    notes = []

    for step in range(3):
        console.print(Panel(f"🔍 Searching iteration {step+1}"))
        search_output = tavily_search(research_question)
        notes.append(search_output)

        prompt = f"""
You previously found:
{search_output}

Summarize insights relevant to:
"{research_question}"
Keep it concise.
"""
        summary = model.generate_content(prompt).text.strip()
        notes.append(summary)
    
    return "\n\n".join(notes)

def final_report(research_question: str, notes: str):
    prompt = f"""
Create a final professional research summary.
Topic: {research_question}
Details collected:
{notes}

Write a clear, structured report.
"""
    return model.generate_content(prompt).text


In [ ]:
user_query = input("Enter your research topic: ")

clarify = clarify_if_needed(user_query)
if "?" in clarify:
    console.print(Panel(f" Clarification needed:\n{clarify}"))
else:
    research_question = produce_research_brief(user_query)
    console.print(Panel(f" Research Question:\n{research_question}"))

    notes = iterative_research(research_question)
    report = final_report(research_question, notes)

    console.print(Panel(" Research Completed", border_style="green"))
    console.print(Markdown(report))


In [13]:
import google.generativeai as genai
from dotenv import load_dotenv
import os
from rich.console import Console
from rich.panel import Panel
from rich.markdown import Markdown

# Load API Key
load_dotenv()
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

# Use Gemini 2.0 Flash
model = genai.GenerativeModel("gemini-2.0-flash")

console = Console()

def clarify_if_needed(user_query: str) -> str:
    prompt = f"""
You are a research assistant. Determine if the user query is specific enough.
If NOT, ask exactly ONE clarifying question.
If yes, return "clear".
User query: "{user_query}"
"""
    response = model.generate_content([prompt]).text
    return response.strip()

def produce_research_brief(user_query: str) -> str:
    prompt = f"""
Convert the topic into a single clear research question:
Topic: "{user_query}"
"""
    return model.generate_content([prompt]).text.strip()

def iterative_research(research_question: str) -> str:
    prompt = f"""
Research the question step-by-step. Provide structured key notes.
Question: {research_question}
"""
    return model.generate_content([prompt]).text.strip()

def final_report(research_question: str, notes: str) -> str:
    prompt = f"""
Write a final research report using the notes.
Question: {research_question}
Notes:
{notes}
"""
    return model.generate_content([prompt]).text.strip()


# ------------------ MAIN FLOW ------------------

user_query = input("Enter your research topic: ")

clarify = clarify_if_needed(user_query)

if clarify.lower() != "clear":
    console.print(Panel(f"Clarification Needed:\n{clarify}", border_style="yellow"))
    
    # ✅ New follow-up step
    user_clarification = input("Your answer: ")

    # Combine original question & clarification to produce a refined question
    refined_query = f"{user_query}. Additional clarification: {user_clarification}"
    research_question = produce_research_brief(refined_query)

    console.print(Panel(f"Refined Research Question:\n{research_question}", border_style="cyan"))

else:
    research_question = produce_research_brief(user_query)
    console.print(Panel(f"Research Question:\n{research_question}", border_style="cyan"))

notes = iterative_research(research_question)
report = final_report(research_question, notes)

console.print(Panel(" Research Completed", border_style="green"))
console.print(Markdown(report))


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Clarification Needed:                                                                                           │
│ Not clear.                                                                                                      │
│                                                                                                                 │
│ What criteria are you using to define "best"?                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Refined Research Question:                                                                                      │
│ What are the best cozy and calm coffee shops in San Francisco for studying?                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│  Research Completed                                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final Research Report: Cozy and Calm Coffee Shops in San Francisco for Studying                  

Question: What are the best cozy and calm coffee shops in San Francisco for studying?                              

Methodology:                                                                                                       

This report identifies potential coffee shops in San Francisco that are conducive to studying based on a           
multi-pronged research approach:                                                                                   

 1 Defining Criteria: Clear definitions were established for "cozy and calm" and "best for studying" based on      
   atmosphere, noise level, Wi-Fi reliability, outlet availability, table space, acceptance of long stays, and     
   potential for crowding.                                                                                         
 2 Online Research: Online searches were conducted using Google, Yelp, TripAdvisor, and San Francisco-focused blogs
   to identify potential candidates.                                                                               
 3 Review Aggregation: Information from multiple sources was compiled and analyzed to identify commonly recommended
   coffee shops.                                                                                                   
 4 Neighborhood Consideration: San Francisco’s diverse neighborhoods were considered, with attention paid to areas 
   known for quieter atmospheres and local populations.                                                            
 5 Attribute Filtering: Reviews were filtered for specific mentions of Wi-Fi, outlets, noise levels, quiet         
   atmosphere, and student-friendly attributes.                                                                    

Findings:                                                                                                          

Based on the research, the following coffee shops are potential candidates for cozy and calm study environments:   

 • Haus Coffee (Inner Sunset): This appears to be the most promising candidate. Located in the residential Inner   
   Sunset, it's likely less crowded and touristy than cafes in more central neighborhoods. Reviews suggest a       
   relaxed and calm atmosphere, making it potentially ideal for focused study sessions.                            
 • Saint Frank Coffee (Multiple Locations): Saint Frank is known for its high-quality coffee and minimalist        
   aesthetic. While not inherently "cozy," the focus on quality suggests attention to detail, including cleanliness
   and service. However, crowding and noise levels may vary by location, requiring further investigation of        
   specific branches.                                                                                              
 • Cafe Trieste (North Beach): A historic Italian cafe with an iconic atmosphere, Cafe Trieste offers a unique and 
   potentially inspiring environment. However, it can be noisy and crowded, particularly during live music         
   performances, making it less suitable for prolonged study sessions requiring intense concentration.             

The following coffee shops were deemed less suitable based on the research:                                        

 • Reveille Coffee Co. (Multiple Locations): More "chic" than "cozy," and prone to noise, especially during peak   
   hours.                                                                                                          
 • Farley's (Multiple Locations): Known for its community vibe, but often crowded and noisy, particularly during   
   peak hours.                                                                                                     
 • The Mill (Divisadero): Known for bread and coffee, but minimalist aesthetic and tendency to be crowded make it  
   less ideal for cozy study sessions.                           